<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jayralencar/pictogram-to-sense/blob/main/CLIP_fine_tuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jayralencar/pictogram-to-sense/blob/main/CLIP_fine_tuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://colab.research.google.com/drive/16xGctJ7bUzkQRGP7QjMGyoWTSbcbpzVD"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# CLIP fine-tuning

This notebook contains the code used for fine-tuning [CLIP](https://github.com/openai/CLIP) for mapping pictograms to WordNet synsets.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Sep 23 00:47:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Download data

The dataset construction is detailed in another [notebook](https://github.com/jayralencar/pictogram-to-sense/blob/main/Download_Dataset.ipynb). You can download the dataset [here](https://drive.google.com/file/d/1P627Qusf8QmaVvrEoevWJNjuUp2luOyd/view) or execute the following cell.

In [ ]:
!gdown https://drive.google.com/uc?id=1P627Qusf8QmaVvrEoevWJNjuUp2luOyd
!unzip data_all.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: data/36684_1.png        
  inflating: data/32276_3.png        
  inflating: data/5800_2.png         
 extracting: data/26606_2.txt        
  inflating: data/30901_1.png        
  inflating: data/2648_1.png         
 extracting: data/5398_4.txt         
 extracting: data/15481_2.txt        
 extracting: data/25750_2.txt        
  inflating: data/9034_1.png         
 extracting: data/32095_4.txt        
  inflating: data/11382_1.txt        
  inflating: data/24348_3.png        
 extracting: data/14769_1.txt        
  inflating: data/11741_1.png        
  inflating: data/31091_2.png        
  inflating: data/5951_2.txt         
  inflating: data/30359_1.txt        
 extracting: data/4702_1.txt         
 extracting: data/8679_3.txt         
 extracting: data/24637_4.txt        
 extracting: data/28043_2.txt        
  inflating: data/29855_1.png        
  inflating: data/8367_1.png         
 extracting: data/2352_1.txt 

## Download requirements

In [ ]:
!git clone -b finetuning https://github.com/jayralencar/train-CLIP

Cloning into 'train-CLIP'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 236 (delta 137), reused 152 (delta 65), pack-reused 0
Receiving objects: 100% (236/236), 232.13 KiB | 1.26 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [ ]:
!pip install pytorch_lightning transformers

     |████████████████████████████████| 924 kB 8.5 MB/s 
     |████████████████████████████████| 2.8 MB 38.7 MB/s 
     |████████████████████████████████| 829 kB 73.1 MB/s 
     |████████████████████████████████| 119 kB 70.4 MB/s 
     |████████████████████████████████| 282 kB 66.1 MB/s 
     |████████████████████████████████| 636 kB 27.9 MB/s 
     |████████████████████████████████| 1.3 MB 66.8 MB/s 
     |████████████████████████████████| 3.3 MB 53.2 MB/s 
     |████████████████████████████████| 895 kB 56.9 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 142 kB 67.6 MB/s 
     |████████████████████████████████| 294 kB 70.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=8c259e2ee587f3f1857ff7d3c0c3bdbf91127beff909cad4ef5eda93e3f40b82
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-afrw2s0h
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-afrw2s0h
     |████████████████████████████████| 64 kB 2.4 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369076 sha256=6db20e608a90c79744c53b1516f0c6c131d20c803f2193e52abeff74e8ed4690
  Stored in directory: /tmp/pip-ephem-wheel-cache-bbwr7eih/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=c8091971cc1b993d7e8461ec8836ea2709b585a90379958b5a2732974c7a159b
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built clip ftfy


In [ ]:
pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'

  Cloning https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup to /tmp/pip-req-build-sy3oxkem
  Running command git clone -q https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup /tmp/pip-req-build-sy3oxkem
  Created wheel for cosine-annealing-warmup: filename=cosine_annealing_warmup-2.0-py3-none-any.whl size=3334 sha256=04c31e494789260c62a023a7f65ce54a28b450ac2b221f0eb7ec2eb7846e5898
  Stored in directory: /tmp/pip-ephem-wheel-cache-84ct9cyc/wheels/6c/b9/45/0fa58a1711c535236d946bbeff05d366eaf6818faed404625e
Successfully built cosine-annealing-warmup


In [ ]:
!mv train-CLIP/ train_clip

## Define Paths

In [ ]:
MODEL_VERSION = "ViT-B/16"
LOGS_PATH = "/content/gdrive/MyDrive/Doutorado/CLIP/logs/"
CHECKPOINTS_PATH = "/content/gdrive/MyDrive/Doutorado/CLIP/checkpoints/"+MODEL_VERSION.replace("/","")
MAX_EPOCHS = 1
BATCH_SIZE = 128
LEARNING_RATE = 6e-05

LOGGER_VERSION = "v3"

# train, test, tune, find_lr
MODE = 'train'

## Define model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
import math
import yaml
import copy
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
import clip

class CLIPFinetuningWrapper(pl.LightningModule):
    # Module based on
    def __init__(self, model_name,batch_size=1, learning_rate=3e-3,):
        super().__init__()
        #hparams
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        # Load CLIP
        self.model, self.preprocess = clip.load(model_name, device=self.device, jit=False)
        
        # Prepare for loss calculating
        self.loss_img = nn.CrossEntropyLoss()
        self.loss_txt = nn.CrossEntropyLoss()

    def forward(self, images, texts):
        return self.model(images, texts)

    def training_step(self, train_batch, idx):
        image, text = train_batch
        
        n = math.ceil(len(image) // self.batch_size)
        image_mbs = torch.chunk(image, n)

        ims = [F.normalize(self.model.encode_image(im), dim=1) for im in image_mbs]
        # gather from all GPUs
        ims = self.all_gather(torch.cat(ims))

        logits_per_image, logits_per_text = self.forward(image,text)
        ground_truth = torch.arange(self.batch_size,dtype=torch.long,device=self.device)

        loss = (self.loss_img(logits_per_image,ground_truth) + self.loss_txt(logits_per_text,ground_truth))/2

        self.log("train_loss", loss, on_epoch=True, prog_bar=True,)

        acc_i = (torch.argmax(logits_per_image, 0) == ground_truth).sum()
        acc_t = (torch.argmax(logits_per_image.t(), 0) == ground_truth).sum()
        acc = (acc_i + acc_t) / 2 / len(image)
        self.log("train_acc", acc, on_epoch=True, prog_bar=True,)
        # acc_i tensor(79, device='cuda:0')
        # acc_t tensor(62, device='cuda:0')
        # acc tensor(0.0043, device='cuda:0')
        return loss


    def validation_step(self, batch, idx):
        image, text = batch

        # print(self.batch_size)
        # /
        if MODE=='tune':
          n = len(image)
        else:
          n = math.ceil(len(image) // self.batch_size)
        # 
        # print(n)
        image_mbs = torch.chunk(image, n)
        
        ims = [F.normalize(self.model.encode_image(im), dim=1) for im in image_mbs]
        # gather from all GPUs
        ims = self.all_gather(torch.cat(ims))

        image_logits, text_logits = self.forward(image, text)
        ground_truth = torch.arange(len(image_logits)).to('cuda:0')
        loss = (F.cross_entropy(image_logits, ground_truth) + F.cross_entropy(text_logits, ground_truth)).div(2)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)

        acc_i = (torch.argmax(image_logits, 0) == ground_truth).sum()
        acc_t = (torch.argmax(image_logits.t(), 0) == ground_truth).sum()
        acc = (acc_i + acc_t) / 2 / len(image)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True,)

    def test_step(self, batch, idx):
        pass

    def configure_optimizers(self):
        lr = self.learning_rate

        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=lr,
            momentum=0.9
        )

        # print(self.num_training_steps)
        # print(int(self.num_training_steps*0.2))
        # Source: https://github.com/openai/CLIP/issues/107
        # Use pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'
        lr_scheduler = CosineAnnealingWarmupRestarts(
            optimizer,
            first_cycle_steps=50,
            cycle_mult=1.0,
            max_lr=lr,
            min_lr=0,
            warmup_steps=int(50*0.1)
        )

        return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
    
    # Sourced from https://github.com/PyTorchLightning/pytorch-lightning/issues/5449
    @property
    def num_training_steps(self) -> int:
        """Total training steps inferred from datamodule and devices."""
        dataset = self.train_dataloader()
        if self.trainer.max_steps:
            return self.trainer.max_steps

        dataset_size = len(dataset.dataset)

        # print("DS:", dataset_size)

        num_devices = max(1, self.trainer.num_gpus, self.trainer.num_processes)
        if self.trainer.tpu_cores:
            num_devices = max(num_devices, self.trainer.tpu_cores)

        effective_batch_size = dataset.batch_size * self.trainer.accumulate_grad_batches * num_devices
        # print("effective_batch_size",effective_batch_size)
        # a = (dataset_size // effective_batch_size) * self.trainer.max_epochs
        a = dataset_size // effective_batch_size
        # print('a',a)
        return a

## Define logger

In [ ]:
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import LearningRateMonitor

tb_logger = pl_loggers.TensorBoardLogger(LOGS_PATH,name=MODEL_VERSION.replace("/",""), version=LOGGER_VERSION)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

## Checkpoints

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINTS_PATH,
    filename=MODEL_VERSION.replace("/","")+'-'+LOGGER_VERSION+'{epoch:02d}-{val_acc:.2f}',
    mode='max',
    monitor="val_acc"
    # save_weights_only=True,
    # save_last=True
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory /content/gdrive/MyDrive/Doutorado/CLIP/checkpoints/ViT-B16 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [ ]:
from pytorch_lightning import Trainer
from train_clip.data.text_image_dm import TextImageDataModule

# BATCH_SIZE = 52
if MODE not in ["resume","test"]:
  model = CLIPFinetuningWrapper("ViT-B/32",batch_size=BATCH_SIZE,learning_rate=LEARNING_RATE)
  dm = TextImageDataModule(folder='./data',batch_size=BATCH_SIZE)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 161MiB/s]


In [ ]:
trainer = Trainer(
    max_epochs=MAX_EPOCHS,
    precision=16,
    gpus=1,
    callbacks=[checkpoint_callback,lr_monitor],
    logger=tb_logger,
    auto_scale_batch_size="binsearch"
)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


## Train

In [ ]:
if MODE == 'train':
  trainer.fit(model,dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | model    | CLIP             | 151 M 
1 | loss_img | CrossEntropyLoss | 0     
2 | loss_txt | CrossEntropyLoss | 0     
----------------------------------------------
151 M     Trainable params
0         Non-trainable params
151 M     Total params
605.109   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
trainer.save_checkpoint(CHECKPOINTS_PATH+"/final.ckpt")

In [ ]:
CHECKPOINTS_PATH+"/final.ckpt"

'/content/gdrive/MyDrive/Doutorado/CLIP/checkpoints/ViT-B16/final.ckpt'

## Tune

In [ ]:
if MODE == 'tune':
  trainer.tune(model, datamodule=dm)

In [ ]:
if MODE == 'find_lr':
  lr_finder = trainer.tuner.lr_find(model,datamodule=dm)

In [ ]:
if MODE == 'find_lr':
  fig = lr_finder.plot(suggest=True)
  fig.show()

In [ ]:
if MODE == 'find_lr':
  print(lr_finder.suggestion())

## Testing

In [ ]:
if MODE == 'test':
  model = CLIPFinetuningWrapper.load_from_checkpoint(CHECKPOINTS_PATH+"/clip-epoch=99-val_loss=1.05.ckpt", model_name=MODEL_VERSION)


In [ ]:
from torchvision import transforms as T
import clip

def fix_img(img):
    return img.convert('RGB') if img.mode != 'RGB' else img

image_transform = T.Compose([
    T.Lambda(fix_img),
    T.RandomResizedCrop(224,
                        scale=(0.75, 1.),
                        ratio=(1., 1.)),
    T.ToTensor(),
    T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

In [ ]:
import torch
from PIL import Image

if MODE == 'test':
  texts = [
    'nocturnal mouselike mammal with forelimbs modified to form membranous wings and anatomical adaptations for echolocation by which they navigate; bat; chiropteran',
  '(baseball) a turn trying to get a hit; he was at bat when it happened; he got four hits in four at-bats; bat; at-bat',
  'a small racket with a long handle used for playing squash; squash_racket; squash_racquet; bat',
  'the club used in playing cricket; a cricket bat has a narrow handle and a broad flat end for hitting; cricket_bat; bat',
  'a club used for hitting a ball in various games; bat',
  'strike with, or as if with a baseball bat; bat the ball; bat',
  "wink briefly; bat one's eyelids; bat; flutter",
  'have a turn at bat; Jones bats first, followed by Martinez; bat',
  "use a bat; Who's batting?; bat",
  'beat thoroughly and conclusively in a competition or fight; We licked the other team on Sunday!; cream; bat; clobber; drub; thrash; lick'
  ]

  with torch.no_grad():
      img_emb = image_transform(Image.open("/content/bat.png")).unsqueeze(0).to('cpu')
      img_enc = model.model.encode_image(img_emb)
      # print(img_enc.size())
      text_emb= clip.tokenize(texts, truncate=True).to('cpu')
      # text_emb = tokenizer(texts,padding='max_length', truncation=True, return_tensors="pt",max_length=77)['input_ids'].to('cpu')
      # text_emb = tokenizer(texts,padding='max_length', truncation=True, return_tensors="pt",max_length=77)
      # print(text_emb)/
      text_enc = model.model.encode_text(text_emb)
      # print(text_enc.size())
      logits_per_image, logits_per_text = model(img_emb,text_emb)
      # print(logits_per_image.size())
      # print(logits_per_text.size())
      probs = logits_per_image.softmax(dim=-1).cpu().numpy()
      # probs = logits_per_image.cpu().numpy()
      better = probs.argmax()
      print(better)
      print(probs)
      print(texts[better])

In [ ]:
trainer = pl.Trainer(
      resume_from_checkpoint=CHECKPOINTS_PATH+"ViT-B16-v0epoch=46-val_acc=0.82.ckpt",
      accelerator='ddp',
      # accelerator='horovod',
      # plugins='ddp_sharded',
      max_epochs=MAX_EPOCHS,
      logger=tb_logger,
      gpus=0,
      callbacks=[checkpoint_callback],
      # precision=16,
      # accumulate_grad_batches=ACCUMULATE_GRAD_BATCHES,
  )

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:792: UserWarning: You are running on single node with no parallelization, so distributed has no effect.
  rank_zero_warn("You are running on single node with no parallelization, so distributed has no effect.")
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1295: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer"
